In [1]:
import numpy as np
import mido
import pretty_midi
import sklearn

In [95]:
def notes_to_instrument(notes, type_inst=0):
    instrument = pretty_midi.Instrument(type_inst)
    instrument.notes = [pretty_midi.Note(100,notes[i],i,i+1) for i in range(len(notes))]
    return instrument

In [3]:
cantus_firmus = np.array([50,53,52,50,55,53,57,55,53,52,50])
melody = pretty_midi.PrettyMIDI()
melody.instruments = [notes_to_instrument(cantus_firmus)]
melody.write("cantus.mid")

In [4]:
counterpoint = np.array([57,57,55,57,59,60,60,59,62,61,62])
melody.instruments = [notes_to_instrument(cantus_firmus), notes_to_instrument(counterpoint)]
melody.write("counterpoint.mid")

In [110]:
def score_counterpoint(cantus_firmus, counterpoint):
    root = cantus_firmus[0]
    perfect_consonances = [0,7]
    imperfect_consonances = [3,4,5,8,9]
    major_sixth = 9
    major_scale = [0,2,4,5,7,9,11]
    
    score = 0
    previous = 0
    previous_cantus = 0
    for i in range(len(cantus_firmus)):
        difference = counterpoint[i] - cantus_firmus[i]
        root_difference = counterpoint[i] - root
        cantus_difference = cantus_firmus[i] - previous_cantus
        previous_note_diff = counterpoint[i] - previous
        if difference < 0:
            score -= 10
        if i == 0 or i == len(cantus_firmus):
            if root_difference%12 == 0:
                score += 10
        elif i == len(cantus_firmus) - 2:
            if difference%12 == major_sixth:
                score += 10
        else:
            if difference%12 in imperfect_consonances and root_difference%12 in major_scale:
                score += 4
            if difference%12 in perfect_consonances and root_difference%12 in major_scale:
                score += 2
            if abs(counterpoint[i]-previous) > 5:
                score -= 5
            if cantus_difference > 0 != previous_note_diff > 0:
                score += 7
            if previous == counterpoint[i]:
                score -= 2
        previous = counterpoint[i]
        previous_cantus = cantus_firmus[i]
    return max(score, 0)

score_counterpoint(cantus_firmus, counterpoint)

37

In [6]:
def mutation(counter, chance):
    mutated = counter.copy()
    for i in range(len(mutated)):
        if np.random.random() < chance:
            mutated[i] += np.random.randint(-4,5)
    return mutated

In [7]:
def crossover(parent1, parent2):
    cutpoint = np.random.randint(0, len(parent1))
    
    child1 = np.concatenate((parent1[:cutpoint], parent2[cutpoint:]), axis=0)
    child2 = np.concatenate((parent2[:cutpoint], parent1[cutpoint:]), axis=0)
    
    return child1, child2

In [52]:
def NormalizeData(data):
    return data/np.sum(data)

In [105]:
def parent_selection(population, cantus_firmus):
    fit = list(map(score_counterpoint,[cantus_firmus for i in range(len(population))], population))
    
    new_pop = []
    new_pop.append(population[fit.index(max(fit))])
    
    fit_sum = sum(fit)
    prob = None
    if fit_sum != 0:
        prob = NormalizeData(fit)
        
    indicies = np.random.choice(len(population), int((len(population)/ 3 - 1)), p = prob)
    
    for i in indicies:
        new_pop.append(population[i])
    
    return new_pop

def breed(parents):
    children = []
    
    #Chooses parents at random and have them make children
    for i in range(len(parents)*2):
        par1 = parents[np.random.choice(len(parents))]
        par2 = parents[np.random.choice(len(parents))]
        
        child1, child2 = crossover(par1, par2)
        
        children.append(child1)
        children.append(child2)
    
    #Go through each child and give them a chance to mutate
    children = list(map(mutation, children, [0.009 for i in range(len(children))]))
    
    return children

def survivor_selection(parents, children, p, cantus_firmus):
    pop = children + parents
    
    fit = list(map(score_counterpoint, [cantus_firmus for i in range(len(pop))], pop))
    most_fit = max(fit)
    best = pop[fit.index(most_fit)]
    
    number_to_keep = 1
    
    new_pop = []
    
    for i in range(number_to_keep):
        new_pop.append(pop[fit.index(max(fit))])
    
    fit_sum = sum(fit)
    prob = None
    if fit_sum != 0:
        prob = NormalizeData(fit)
    
    indicies = np.random.choice(len(pop), p-number_to_keep, p = prob)
    
    for i in indicies:
        new_pop.append(pop[i])
        
    return new_pop, best
    
def evo(cantus_firmus, popsize, iterations, low_note, high_note):
    population = []
    best = None
    
    for i in range(popsize):
        population.append(np.random.randint(low_note, high_note, len(cantus_firmus)))
    
    for i in range(iterations):
        parents = parent_selection(population, cantus_firmus)
        children = breed(parents)
    
        population, best = survivor_selection(parents, children, popsize, cantus_firmus)
    
    return best


best = evo(cantus_firmus, 100, 500, 52, 68)

In [107]:
melody.instruments = [notes_to_instrument(cantus_firmus), notes_to_instrument(best, 5)]
melody.write("evo_counterpoint.mid")

In [106]:
third_voice = evo(best, 100, 500, 64, 80)
melody.instruments = [notes_to_instrument(cantus_firmus), notes_to_instrument(best, 5), notes_to_instrument(third_voice, 10)]
melody.write("three_voice_evo_counterpoint.mid")

In [109]:
fourth_voice = evo(third_voice, 100, 500, 76, 94)
melody.instruments = [notes_to_instrument(cantus_firmus), notes_to_instrument(best, 5), notes_to_instrument(third_voice, 10), notes_to_instrument(fourth_voice, 3)]
melody.write("four_voice_evo_counterpoint.mid")